In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
import numpy as np

def etl_cof(filename):
    df = pd.read_csv(filename)
    df.drop(['Posted Date','Card No.','Credit','Category'],axis=1,inplace=True)
    df.dropna(inplace=True)
    df.rename(index=str, columns={"Transaction Date": "Date","Debit" : "Amount"}, inplace=True)
    #print df
    return df

def etl_boa(filename):
    df = pd.read_csv(filename,skiprows=6)
    #this is going to be hard coded
    #df.drop(range(1,49), axis=0, inplace=True)
    
    df.dropna(inplace=True)
    df.drop('Running Bal.',axis=1,inplace=True)
    c1 = 'CAPITAL ONE DES:MOBILE PMT'
    income = df[df['Amount'] > 0]
    df = df[df['Amount'] < 0]
    #df[df["col"].str.contains('this|that')==False]

    df = df[df['Description'].str.contains(c1)==False]
    df['Amount'] *= -1
    return income, df
def applyLabels(df,filename):
    categories = pd.read_csv(filename)
    categories.drop_duplicates(subset='Location', inplace=True)
    df['Category'] = ''
    i = 0
    print set(categories['Category'].tolist())
    for index, row in df.iterrows():
        if row['Description'] in categories['Location'].tolist():
            category = categories.loc[categories['Location']==row['Description'],'Category'].tolist()[0]
            df.loc[index,'Category'] = category
        else:
            #print row['Description'] + " not found. Would you like to input it manually?"
            cat = raw_input(row['Description'] + " not found. Add it manually or press enter to skip ")
            if cat:
                df.loc[index,'Category'] = cat
                temp = pd.DataFrame(data={'Location' : [row['Description']], 'Category' : [cat]})
                categories = pd.concat([categories,temp],axis=0)                        
        i += 1
        
    categories.to_csv(filename,index=False)
    return df
def getSpendingByCategory(df):
    categories = set(df['Category'].tolist())
    sbc = pd.DataFrame(columns = ['Category','Total'])
    
    i = 0
    totalSpending = sum(df.loc[:,'Amount'].tolist())
    for c in categories:
        temp = df.loc[df['Category'] == c]
        total = sum(temp.loc[:,'Amount'].tolist())
        percent = str(round(total/totalSpending * 100,2)) + "%"
        entry = pd.DataFrame(data={'Category' : [c], 'Total' : [total], 'Percent of Spending' : [percent]})
        #print entry
        sbc = pd.concat([sbc,entry],axis=0)
        i += 1
    #Bandaid fix
    totalSpending -= sbc.loc[:,'Category'] == ''
    totalSpending = totalSpending.iloc[0]
    sbc = sbc[sbc['Category'] != '']
    sbc.sort_values('Total',inplace=True,ascending=False)
    sbc.to_csv('spending_by_category.csv', index=False)
    
    return (sbc, totalSpending)

def plotSBC(sbc):
    #Bars
    plt.figure(1)
    bar(sbc.loc[:,'Total'],sbc.loc[:,'Category'],'Spending by Category','Category','Spending',sbc.loc[:,'Percent of Spending'])

def plotSPD(both):
    spd = both.groupby('Date',axis=0).sum().reset_index()
    display(spd)
    plt.figure(2)
    labels = spd.loc[:,'Date'].apply(lambda x: str(x)[0:10])
    bar(spd.loc[:,'Amount'],labels,'Spending per Day','Date','Spending')
    
    spending = spd.loc[:,'Amount'].values.tolist()
    print 'Mean & Median spending per day: ' + str(round(np.mean(spending),2)) + ' ' + str(np.median(spending))

def bar(yvals, xlabels, title, xtitle, ytitle, ylabels = None):
    fig, ax = plt.subplots(figsize=(9, 7))
    plt.bar(range(len(yvals)), yvals)
    plt.xticks(range(len(yvals)), xlabels, rotation='vertical')
    if ylabels is not None:
        rects = ax.patches
        for rect, label in zip(rects, ylabels):
            height = rect.get_height()
            ax.text(rect.get_x() + rect.get_width() / 2, height + 5, label,
                ha='center', va='bottom')
    plt.title(title)
    plt.xlabel(xtitle)
    plt.ylabel(ytitle)
    plt.tight_layout()
    plt.show()

def stacked_bar(data, series_labels, category_labels=None, 
                show_values=False, value_format="{}", y_label=None, 
                grid=True, reverse=False):
    """Plots a stacked bar chart with the data and labels provided.

    Keyword arguments:
    data            -- 2-dimensional numpy array or nested list
                       containing data for each series in rows
    series_labels   -- list of series labels (these appear in
                       the legend)
    category_labels -- list of category labels (these appear
                       on the x-axis)
    show_values     -- If True then numeric value labels will 
                       be shown on each bar
    value_format    -- Format string for numeric value labels
                       (default is "{}")
    y_label         -- Label for y-axis (str)
    grid            -- If True display grid
    reverse         -- If True reverse the order that the
                       series are displayed (left-to-right
                       or right-to-left)
    """

    ny = len(data[0])
    ind = list(range(ny))

    axes = []
    cum_size = np.zeros(ny)

    data = np.array(data)

    if reverse:
        data = np.flip(data, axis=1)
        category_labels = reversed(category_labels)

    for i, row_data in enumerate(data):
        axes.append(plt.bar(ind, row_data, bottom=cum_size, 
                            label=series_labels[i]))
        cum_size += row_data

    if category_labels:
        plt.xticks(ind, category_labels)

    if y_label:
        plt.ylabel(y_label)

    plt.legend()

def main():
    #ETL
    cofData = etl_cof('2018-09-30_transaction_download.csv')
    income, boaData = etl_boa('stmt_september.csv')
    both = pd.concat([cofData, boaData])
    both["Date"] = pd.to_datetime(both["Date"])
    both.sort_values(by="Date", inplace=True, ascending=True)
    #Apply Labels
    both = applyLabels(both,"categories.csv")
    #Write to CSV
    both.to_csv('both.csv')
    
    sbc, totalSpending = getSpendingByCategory(both)
    
    plotSBC(sbc)
    
    #Print total spending
    total = pd.DataFrame(data={'Category' : ['Total'], 'Percent of Spending'  : [''], 'Total' : [totalSpending]})
    sbc = pd.concat([sbc,total],axis=0)
    display(sbc)
    totalIncome = sum(income.loc[:,'Amount'].tolist())
    netIncome = totalIncome-totalSpending
    print 'Income: ' + str(totalIncome) + ' Spending: ' + str(totalSpending) + ' Net Income: ' + str(netIncome)
      
    #plotSPD(both)
    
    #stacked bar graph for the month
    #stacked_bar(sbc['Total'],sbc['Category'], category_labels=None, value_format="{}", y_label=None, reverse=False)
if __name__ == '__main__':
    main()

In [ ]:
    #Pie
    #fig1, ax1 = plt.subplots()
    #ax1.pie(x=sbc.loc[:,'Total'],labels=sbc.loc[:,'Category'],autopct='%1.1f%%',radius=5.0)
    # Equal aspect ratio ensures that pie is drawn as a circle
    #ax1.axis('equal')